In [38]:
from sympy import *
from sympy.vector import *
import regex

pax, pay, pbx, pby, vax, vay, vbx, vby, aax, aay, abx, aby, pix, piy, vfx, vfy, jx, jy = symbols('p_a_x p_a_y p_b_x p_b_y v_a_x v_a_y v_b_x v_b_y a_a_x a_a_y a_b_x a_b_y p_i_x p_i_y v_f_x v_f_y j_x j_y',real=True)
F, T, Ta, Tb = symbols('F T T_a T_b',real=True,positive=True)


def gdcode(expr):
    expr = collect_const(simplify(expr,1.0))
    rhs = ccode(expr)
    opc = count_ops(expr)
    # turn squares into multiplication
    rhs = regex.sub(r'pow\(\s*(\w+)\s*,\s*2\s*\)', r'(\1*\1)', rhs)
    rhs = regex.sub(r'pow\(\s*(\w+)\s*,\s*-2\s*\)', r'(1.0/(\1*\1))', rhs)
    # turn cubes into multiplication
    rhs = regex.sub(r'pow\(\s*(\w+)\s*,\s*3\s*\)', r'(\1*\1*\1)', rhs)
    rhs = regex.sub(r'pow\(\s*(\w+)\s*,\s*-3\s*\)', r'(1.0/(\1*\1*\1))', rhs)
    # evaluate fractions
    rhs = regex.sub(r'(\d+\.?\d*)/(\d+\.?\d*)', lambda match: str(float(match.group(1))/float(match.group(2))), rhs)
    # convert integers to floats
    rhs = regex.sub(r'(?<!\.)\b(\d+)\b(?!\.)',r'\1.0',rhs)
    # rename vector parts
    rhs = regex.sub(r'\b(\w+)_(x|y)',r'\1.\2',rhs)
    # fix ternary if
    rhs = regex.sub(r'\(\s*\(\s*(.*?)\s*\)\s*\?\s*\(\s*\n\s*(.*?)\s*\n\s*\)\s*\n\s*:\s*\(\s*\n\s*(.*?)\s*\n\s*\)\s*\)', r'\2 if \1 else \3', rhs)
    return (rhs,opc)

def make_code(qs):
    cse_qs = cse(qs,symbols=numbered_symbols('_'))
    code=''
    opcount=0
    for x in cse_qs[0]:
        rhs, opc = gdcode(x[1])
        opcount+=opc
        code+='var '+str(x[0])+':float = '+rhs
        code+='\n'

    for x in cse_qs[1]:
        rhs, opc = gdcode(x)
        opcount+=opc
        code+=rhs
        code+='\n'
    
    code = '# opcount: '+str(opcount)+'\n'+code

    return code


In [39]:
# linear-linear intercept
q_pix = Eq(vax*T + pax, vbx*T + pbx)
q_piy = Eq(vay*T + pay, vby*T + pby)
display(q_pix, q_piy)

q_f = Eq(F**2, vax**2 + vay**2)
display(q_f)

q_vax = Eq(vax,solve(q_pix, vax)[0])
q_vay = Eq(vay,solve(q_piy, vay)[0])
display(q_vax, q_vay)

q_f2 = q_f.subs({vax:q_vax.rhs, vay:q_vay.rhs})
display(q_f2)
q_f2 = Eq(q_f2.lhs * 4*T**2, q_f2.rhs * 4*T**2)
q_f2 = simplify(q_f2)
q_f2 = q_f2.lhs - q_f2.rhs
display(q_f2)

q_f2 = collect(expand(q_f2),T)
display(q_f2)
q_f2_terms = collect(expand(q_f2),T, evaluate=False)
ca, cb, cc, cd, ce = symbols('c_a c_b c_c c_d c_e', real=True)
#q_ca = Eq(ca, q_f2_terms[T**4])
#q_cb = Eq(cb, q_f2_terms[T**3])
q_cc = Eq(cc, q_f2_terms[T**2])
q_cd = Eq(cd, q_f2_terms[T**1])
q_ce = Eq(ce, q_f2_terms[1])
display(q_cc, q_cd, q_ce)
q_f2 = q_f2.subs({q_cc.rhs:q_cc.lhs, q_cd.rhs:q_cd.lhs, q_ce.rhs:q_ce.lhs})
display(q_f2)

q_t = solve(q_f2, T)
display(q_t)

sols = [q.subs({q_cc.lhs:q_cc.rhs, q_cd.lhs:q_cd.rhs, q_ce.lhs:q_ce.rhs}) for q in q_t]
print(make_code(sols))

Eq(T*v_a_x + p_a_x, T*v_b_x + p_b_x)

Eq(T*v_a_y + p_a_y, T*v_b_y + p_b_y)

Eq(F**2, v_a_x**2 + v_a_y**2)

Eq(v_a_x, (T*v_b_x - p_a_x + p_b_x)/T)

Eq(v_a_y, (T*v_b_y - p_a_y + p_b_y)/T)

...

Eq(c_d, 8*p_a_x*v_b_x + 8*p_a_y*v_b_y - 8*p_b_x*v_b_x - 8*p_b_y*v_b_y)

Eq(c_e, -4*p_a_x**2 + 8*p_a_x*p_b_x - 4*p_a_y**2 + 8*p_a_y*p_b_y - 4*p_b_x**2 - 4*p_b_y**2)

T**2*c_c + T*c_d + c_e

[(-c_d - sqrt(-4*c_c*c_e + c_d**2))/(2*c_c),
 (-c_d + sqrt(-4*c_c*c_e + c_d**2))/(2*c_c)]

# opcount: 44
var _0:float = 8.0*v_b.x
var _1:float = _0*p_a.x
var _2:float = 8.0*v_b.y
var _3:float = _2*p_a.y
var _4:float = _0*p_b.x
var _5:float = _2*p_b.y
var _6:float = _1 + _3 - (_4 + _5)
var _7:float = 4.0*((F*F) - (v_b.x*v_b.x) - (v_b.y*v_b.y))
var _8:float = sqrt((_6*_6) + 16.0*_7*((p_a.x*p_a.x) - 2.0*p_a.x*p_b.x + (p_a.y*p_a.y) - 2.0*p_a.y*p_b.y + (p_b.x*p_b.x) + (p_b.y*p_b.y)))
var _9:float = (0.5)/_7
-_9*(_6 + _8)
_9*(-_1 - _3 + _4 + _5 + _8)



In [40]:
# linear-quadratic intercept
q_pix = Eq(vax*T + pax, abx*(T**2)/2 + vbx*T + pbx)
q_piy = Eq(vay*T + pay, aby*(T**2)/2 + vby*T + pby)
display(q_pix, q_piy)

q_f = Eq(F**2, vax**2 + vay**2)
display(q_f)

q_vax = Eq(vax,solve(q_pix, vax)[0])
q_vay = Eq(vay,solve(q_piy, vay)[0])
display(q_vax, q_vay)

q_f2 = q_f.subs({vax:q_vax.rhs, vay:q_vay.rhs})
display(q_f2)
q_f2 = Eq(q_f2.lhs * 4*T**2, q_f2.rhs * 4*T**2)
q_f2 = simplify(q_f2)
q_f2 = q_f2.lhs - q_f2.rhs
display(q_f2)

q_f2 = collect(expand(q_f2),T)
display(q_f2)
q_f2_terms = collect(expand(q_f2),T, evaluate=False)
ca, cb, cc, cd, ce = symbols('c_a c_b c_c c_d c_e', real=True)
q_ca = Eq(ca, q_f2_terms[T**4])
q_cb = Eq(cb, q_f2_terms[T**3])
q_cc = Eq(cc, q_f2_terms[T**2])
q_cd = Eq(cd, q_f2_terms[T**1])
q_ce = Eq(ce, q_f2_terms[1])
display(q_ca, q_cb, q_cc, q_cd, q_ce)
q_f2 = q_f2.subs({q_ca.rhs:q_ca.lhs, q_cb.rhs:q_cb.lhs, q_cc.rhs:q_cc.lhs, q_cd.rhs:q_cd.lhs, q_ce.rhs:q_ce.lhs})
display(q_f2)

q_t = solve(q_f2, T)
display(q_t)

sols = [q.subs({q_ca.lhs:q_ca.rhs, q_cb.lhs:q_cb.rhs, q_cc.lhs:q_cc.rhs, q_cd.lhs:q_cd.rhs, q_ce.lhs:q_ce.rhs}) for q in q_t]
print(make_code(sols))

Eq(T*v_a_x + p_a_x, T**2*a_b_x/2 + T*v_b_x + p_b_x)

Eq(T*v_a_y + p_a_y, T**2*a_b_y/2 + T*v_b_y + p_b_y)

Eq(F**2, v_a_x**2 + v_a_y**2)

Eq(v_a_x, T*a_b_x/2 + v_b_x - p_a_x/T + p_b_x/T)

Eq(v_a_y, T*a_b_y/2 + v_b_y - p_a_y/T + p_b_y/T)

...

Eq(c_d, 8*p_a_x*v_b_x + 8*p_a_y*v_b_y - 8*p_b_x*v_b_x - 8*p_b_y*v_b_y)

Eq(c_e, -4*p_a_x**2 + 8*p_a_x*p_b_x - 4*p_a_y**2 + 8*p_a_y*p_b_y - 4*p_b_x**2 - 4*p_b_y**2)

T**4*c_a + T**3*c_b + T**2*c_c + T*c_d + c_e

[Piecewise((-sqrt(-2*(-(c_c/c_a - 3*c_b**2/(8*c_a**2))**3/108 + (c_c/c_a - 3*c_b**2/(8*c_a**2))*(c_e/c_a - c_b*c_d/(4*c_a**2) + c_b**2*c_c/(16*c_a**3) - 3*c_b**4/(256*c_a**4))/3 - (c_d/c_a - c_b*c_c/(2*c_a**2) + c_b**3/(8*c_a**3))**2/8)**(1/3) - 2*c_c/(3*c_a) + c_b**2/(4*c_a**2))/2 - sqrt((2*c_d/c_a - c_b*c_c/c_a**2 + c_b**3/(4*c_a**3))/sqrt(-2*(-(c_c/c_a - 3*c_b**2/(8*c_a**2))**3/108 + (c_c/c_a - 3*c_b**2/(8*c_a**2))*(c_e/c_a - c_b*c_d/(4*c_a**2) + c_b**2*c_c/(16*c_a**3) - 3*c_b**4/(256*c_a**4))/3 - (c_d/c_a - c_b*c_c/(2*c_a**2) + c_b**3/(8*c_a**3))**2/8)**(1/3) - 2*c_c/(3*c_a) + c_b**2/(4*c_a**2)) + 2*(-(c_c/c_a - 3*c_b**2/(8*c_a**2))**3/108 + (c_c/c_a - 3*c_b**2/(8*c_a**2))*(c_e/c_a - c_b*c_d/(4*c_a**2) + c_b**2*c_c/(16*c_a**3) - 3*c_b**4/(256*c_a**4))/3 - (c_d/c_a - c_b*c_c/(2*c_a**2) + c_b**3/(8*c_a**3))**2/8)**(1/3) - 4*c_c/(3*c_a) + c_b**2/(2*c_a**2))/2 - c_b/(4*c_a), Eq(c_e/c_a - c_b*c_d/(4*c_a**2) + c_c**2/(12*c_a**2), 0)), (-sqrt(-2*(-c_e/c_a + c_b*c_d/(4*c_a**2) - c_c**2/(12

# opcount: 192
var _0:float = -((a_b.x*a_b.x) + (a_b.y*a_b.y))
var _1:float = (1.0/(_0*_0))
var _2:float = 4.0*a_b.x
var _3:float = 4.0*a_b.y
var _4:float = -(_2*v_b.x + _3*v_b.y)
var _5:float = (_4*_4)
var _6:float = _1*_5
var _7:float = 1.0/_0
var _8:float = 4.0*((F*F) + a_b.x*p_a.x + a_b.y*p_a.y) - (_2*p_b.x + _3*p_b.y + 4.0*(v_b.x*v_b.x) + 4.0*(v_b.y*v_b.y))
var _9:float = _7*_8
var _10:float = -0.375*_6 + _9
var _11:float = (_10*_10*_10)
var _12:float = (1.0/(_0*_0*_0))
var _13:float = _12*(_4*_4*_4)
var _14:float = 8.0*v_b.x
var _15:float = 8.0*v_b.y
var _16:float = _14*p_a.x + _15*p_a.y - (_14*p_b.x + _15*p_b.y)
var _17:float = _16*_7
var _18:float = _1*_4
var _19:float = _18*_8
var _20:float = pow((0.125)*_13 + _17 - 0.5*_19, 2.0)
var _21:float = -0.25*(_16*_18 + 16.0*_7*((p_a.x*p_a.x) - 2.0*p_a.x*p_b.x + (p_a.y*p_a.y) - 2.0*p_a.y*p_b.y + (p_b.x*p_b.x) + (p_b.y*p_b.y)))
var _22:float = (0.0625)*_12*_5*_8 + _21 - 0.01171875*pow(_4, 4.0)/pow(_0, 4.0)
var _23:float = (0.3333333333

In [41]:
# quadratic-linear intercept

q_pix = Eq(aax*(T**2)/2 + vax*T + pax, vbx*T + pbx)
q_piy = Eq(aay*(T**2)/2 + vay*T + pay, vby*T + pby)
display(q_pix, q_piy)

q_f = Eq(F**2, aax**2 + aay**2)
display(q_f)

q_aax = Eq(aax,solve(q_pix, aax)[0])
q_aay = Eq(aay,solve(q_piy, aay)[0])
display(q_aax, q_aay)

q_f2 = q_f.subs({aax:q_aax.rhs, aay:q_aay.rhs})
q_f2 = Eq(q_f2.lhs * T**4, q_f2.rhs * T**4)
q_f2 = simplify(q_f2)
q_f2 = q_f2.lhs - q_f2.rhs
display(q_f2)

q_f2 = collect(expand(q_f2),T)
display(q_f2)
q_f2_terms = collect(expand(q_f2),T, evaluate=False)
ca, cb, cc, cd, ce = symbols('c_a c_b c_c c_d c_e', real=True)
q_ca = Eq(ca, q_f2_terms[T**4])
#q_cb = Eq(cb, q_f2_terms[T**3])
q_cc = Eq(cc, q_f2_terms[T**2])
q_cd = Eq(cd, q_f2_terms[T**1])
q_ce = Eq(ce, q_f2_terms[1])
display(q_ca, q_cc, q_cd, q_ce)
q_f2 = q_f2.subs({q_ca.rhs:q_ca.lhs, q_cc.rhs:q_cc.lhs, q_cd.rhs:q_cd.lhs, q_ce.rhs:q_ce.lhs})
display(q_f2)

q_t = solve(q_f2, T)
display(q_t)

sols = [q.subs({q_ca.lhs:q_ca.rhs, q_cb.lhs:q_cb.rhs, q_cc.lhs:q_cc.rhs, q_cd.lhs:q_cd.rhs, q_ce.lhs:q_ce.rhs}) for q in q_t]
print(make_code(sols))

Eq(T**2*a_a_x/2 + T*v_a_x + p_a_x, T*v_b_x + p_b_x)

Eq(T**2*a_a_y/2 + T*v_a_y + p_a_y, T*v_b_y + p_b_y)

Eq(F**2, a_a_x**2 + a_a_y**2)

Eq(a_a_x, 2*(T*(-v_a_x + v_b_x) - p_a_x + p_b_x)/T**2)

Eq(a_a_y, 2*(T*(-v_a_y + v_b_y) - p_a_y + p_b_y)/T**2)

...

Eq(c_d, -8*p_a_x*v_a_x + 8*p_a_x*v_b_x - 8*p_a_y*v_a_y + 8*p_a_y*v_b_y + 8*p_b_x*v_a_x - 8*p_b_x*v_b_x + 8*p_b_y*v_a_y - 8*p_b_y*v_b_y)

Eq(c_e, -4*p_a_x**2 + 8*p_a_x*p_b_x - 4*p_a_y**2 + 8*p_a_y*p_b_y - 4*p_b_x**2 - 4*p_b_y**2)

T**4*c_a + T**2*c_c + T*c_d + c_e

[Piecewise((-sqrt(-2*(c_c*c_e/(3*c_a**2) - c_d**2/(8*c_a**2) - c_c**3/(108*c_a**3))**(1/3) - 2*c_c/(3*c_a))/2 - sqrt(2*(c_c*c_e/(3*c_a**2) - c_d**2/(8*c_a**2) - c_c**3/(108*c_a**3))**(1/3) - 4*c_c/(3*c_a) + 2*c_d/(c_a*sqrt(-2*(c_c*c_e/(3*c_a**2) - c_d**2/(8*c_a**2) - c_c**3/(108*c_a**3))**(1/3) - 2*c_c/(3*c_a))))/2, Eq(c_e/c_a + c_c**2/(12*c_a**2), 0)), (-sqrt(-2*(-c_e/c_a - c_c**2/(12*c_a**2))/(3*(sqrt((-c_e/c_a - c_c**2/(12*c_a**2))**3/27 + (c_c*c_e/(3*c_a**2) - c_d**2/(8*c_a**2) - c_c**3/(108*c_a**3))**2/4) - c_c*c_e/(6*c_a**2) + c_d**2/(16*c_a**2) + c_c**3/(216*c_a**3))**(1/3)) + 2*(sqrt((-c_e/c_a - c_c**2/(12*c_a**2))**3/27 + (c_c*c_e/(3*c_a**2) - c_d**2/(8*c_a**2) - c_c**3/(108*c_a**3))**2/4) - c_c*c_e/(6*c_a**2) + c_d**2/(16*c_a**2) + c_c**3/(216*c_a**3))**(1/3) - 2*c_c/(3*c_a))/2 - sqrt(2*(-c_e/c_a - c_c**2/(12*c_a**2))/(3*(sqrt((-c_e/c_a - c_c**2/(12*c_a**2))**3/27 + (c_c*c_e/(3*c_a**2) - c_d**2/(8*c_a**2) - c_c**3/(108*c_a**3))**2/4) - c_c*c_e/(6*c_a**2) + c_d**2/(16*c_a**2) 

# opcount: 145
var _0:float = (1.0/(F*F))
var _1:float = 8.0*(v_a.x*v_b.x + v_a.y*v_b.y) - 4.0*((v_a.x*v_a.x) + (v_a.y*v_a.y) + (v_b.x*v_b.x) + (v_b.y*v_b.y))
var _2:float = _0*_1
var _3:float = (0.6666666666666666)*_2
var _4:float = (_1*_1*_1)/pow(F, 6.0)
var _5:float = pow(F, -4.0)
var _6:float = 8.0*(-p_a.x*v_a.x + p_a.x*v_b.x - p_a.y*v_a.y + p_a.y*v_b.y + p_b.x*v_a.x - p_b.x*v_b.x + p_b.y*v_a.y - p_b.y*v_b.y)
var _7:float = _5*(_6*_6)
var _8:float = 8.0*(p_a.x*p_b.x + p_a.y*p_b.y) - 4.0*((p_a.x*p_a.x) + (p_a.y*p_a.y) + (p_b.x*p_b.x) + (p_b.y*p_b.y))
var _9:float = (0.3333333333333333)*_1*_5*_8 - 0.009259259259259259*_4 - 0.125*_7
var _10:float = 2.0*cbrt(_9)
var _11:float = sqrt(-_10 - _3)
var _12:float = (0.5)*_11
var _13:float = (1.3333333333333333)*_2
var _14:float = -_13
var _15:float = 2.0*_0*_6
var _16:float = _15/_11
var _17:float = (0.5)*sqrt(_10 + _14 + _16)
var _18:float = _0*_8 + (0.08333333333333333)*(_1*_1)*_5
var _19:float = _18 == 0.0
var _20:float = -_18
var _21:flo

In [42]:
# quadratic-quadratic intercept

q_pix = Eq(aax*(T**2)/2 + vax*T + pax, abx*(T**2)/2 + vbx*T + pbx)
q_piy = Eq(aay*(T**2)/2 + vay*T + pay, aby*(T**2)/2 + vby*T + pby)
display(q_pix, q_piy)

q_f = Eq(F**2, aax**2 + aay**2)
display(q_f)

q_aax = Eq(aax,solve(q_pix, aax)[0])
q_aay = Eq(aay,solve(q_piy, aay)[0])
display(q_aax, q_aay)

q_f2 = q_f.subs({aax:q_aax.rhs, aay:q_aay.rhs})
q_f2 = Eq(q_f2.lhs * T**4, q_f2.rhs * T**4)
q_f2 = simplify(q_f2)
q_f2 = q_f2.lhs - q_f2.rhs
display(q_f2)

q_f2 = collect(expand(q_f2),T)
display(q_f2)
q_f2_terms = collect(expand(q_f2),T, evaluate=False)
ca, cb, cc, cd, ce = symbols('c_a c_b c_c c_d c_e', real=True)
q_ca = Eq(ca, q_f2_terms[T**4])
q_cb = Eq(cb, q_f2_terms[T**3])
q_cc = Eq(cc, q_f2_terms[T**2])
q_cd = Eq(cd, q_f2_terms[T**1])
q_ce = Eq(ce, q_f2_terms[1])
display(q_ca, q_cb, q_cc, q_cd, q_ce)
q_f2 = q_f2.subs({q_ca.rhs:q_ca.lhs, q_cb.rhs:q_cb.lhs, q_cc.rhs:q_cc.lhs, q_cd.rhs:q_cd.lhs, q_ce.rhs:q_ce.lhs})
display(q_f2)

q_t = solve(q_f2, T)
display(q_t)

sols = [q.subs({q_ca.lhs:q_ca.rhs, q_cb.lhs:q_cb.rhs, q_cc.lhs:q_cc.rhs, q_cd.lhs:q_cd.rhs, q_ce.lhs:q_ce.rhs}) for q in q_t]
print(make_code(sols))

Eq(T**2*a_a_x/2 + T*v_a_x + p_a_x, T**2*a_b_x/2 + T*v_b_x + p_b_x)

Eq(T**2*a_a_y/2 + T*v_a_y + p_a_y, T**2*a_b_y/2 + T*v_b_y + p_b_y)

Eq(F**2, a_a_x**2 + a_a_y**2)

Eq(a_a_x, (T**2*a_b_x + 2*T*(-v_a_x + v_b_x) - 2*p_a_x + 2*p_b_x)/T**2)

Eq(a_a_y, (T**2*a_b_y + 2*T*(-v_a_y + v_b_y) - 2*p_a_y + 2*p_b_y)/T**2)

...

Eq(c_d, -8*p_a_x*v_a_x + 8*p_a_x*v_b_x - 8*p_a_y*v_a_y + 8*p_a_y*v_b_y + 8*p_b_x*v_a_x - 8*p_b_x*v_b_x + 8*p_b_y*v_a_y - 8*p_b_y*v_b_y)

Eq(c_e, -4*p_a_x**2 + 8*p_a_x*p_b_x - 4*p_a_y**2 + 8*p_a_y*p_b_y - 4*p_b_x**2 - 4*p_b_y**2)

T**4*c_a + T**3*c_b + T**2*c_c + T*c_d + c_e

[Piecewise((-sqrt(-2*(-(c_c/c_a - 3*c_b**2/(8*c_a**2))**3/108 + (c_c/c_a - 3*c_b**2/(8*c_a**2))*(c_e/c_a - c_b*c_d/(4*c_a**2) + c_b**2*c_c/(16*c_a**3) - 3*c_b**4/(256*c_a**4))/3 - (c_d/c_a - c_b*c_c/(2*c_a**2) + c_b**3/(8*c_a**3))**2/8)**(1/3) - 2*c_c/(3*c_a) + c_b**2/(4*c_a**2))/2 - sqrt((2*c_d/c_a - c_b*c_c/c_a**2 + c_b**3/(4*c_a**3))/sqrt(-2*(-(c_c/c_a - 3*c_b**2/(8*c_a**2))**3/108 + (c_c/c_a - 3*c_b**2/(8*c_a**2))*(c_e/c_a - c_b*c_d/(4*c_a**2) + c_b**2*c_c/(16*c_a**3) - 3*c_b**4/(256*c_a**4))/3 - (c_d/c_a - c_b*c_c/(2*c_a**2) + c_b**3/(8*c_a**3))**2/8)**(1/3) - 2*c_c/(3*c_a) + c_b**2/(4*c_a**2)) + 2*(-(c_c/c_a - 3*c_b**2/(8*c_a**2))**3/108 + (c_c/c_a - 3*c_b**2/(8*c_a**2))*(c_e/c_a - c_b*c_d/(4*c_a**2) + c_b**2*c_c/(16*c_a**3) - 3*c_b**4/(256*c_a**4))/3 - (c_d/c_a - c_b*c_c/(2*c_a**2) + c_b**3/(8*c_a**3))**2/8)**(1/3) - 4*c_c/(3*c_a) + c_b**2/(2*c_a**2))/2 - c_b/(4*c_a), Eq(c_e/c_a - c_b*c_d/(4*c_a**2) + c_c**2/(12*c_a**2), 0)), (-sqrt(-2*(-c_e/c_a + c_b*c_d/(4*c_a**2) - c_c**2/(12

# opcount: 214
var _0:float = (F*F) - ((a_b.x*a_b.x) + (a_b.y*a_b.y))
var _1:float = (1.0/(_0*_0))
var _2:float = 4.0*a_b.x
var _3:float = 4.0*a_b.y
var _4:float = _2*v_a.x + _3*v_a.y - (_2*v_b.x + _3*v_b.y)
var _5:float = (_4*_4)
var _6:float = _1*_5
var _7:float = 1.0/_0
var _8:float = 4.0*(a_b.x*p_a.x + a_b.y*p_a.y) + 8.0*(v_a.x*v_b.x + v_a.y*v_b.y) - (_2*p_b.x + _3*p_b.y + 4.0*(v_a.x*v_a.x) + 4.0*(v_a.y*v_a.y) + 4.0*(v_b.x*v_b.x) + 4.0*(v_b.y*v_b.y))
var _9:float = _7*_8
var _10:float = -0.375*_6 + _9
var _11:float = (_10*_10*_10)
var _12:float = (1.0/(_0*_0*_0))
var _13:float = _12*(_4*_4*_4)
var _14:float = 8.0*(-p_a.x*v_a.x + p_a.x*v_b.x - p_a.y*v_a.y + p_a.y*v_b.y + p_b.x*v_a.x - p_b.x*v_b.x + p_b.y*v_a.y - p_b.y*v_b.y)
var _15:float = _14*_7
var _16:float = _1*_4
var _17:float = _16*_8
var _18:float = pow((0.125)*_13 + _15 - 0.5*_17, 2.0)
var _19:float = -0.25*(_14*_16 + 16.0*_7*((p_a.x*p_a.x) - 2.0*p_a.x*p_b.x + (p_a.y*p_a.y) - 2.0*p_a.y*p_b.y + (p_b.x*p_b.x) + (p_b.y*p_b.y))